In [16]:
import numpy
import pandas as pd 
import jieba
import jieba.analyse
import os

#单个字符 全角转半角
def Q2B(uchar):
    """单个字符 全角转半角"""
    inside_code = ord(uchar)
    if inside_code == 0x3000:
        inside_code = 0x0020
    else:
        inside_code -= 0xfee0
    if inside_code < 0x0020 or inside_code > 0x7e: #转完之后不是半角字符返回原来的字符
        return uchar
    return chr(inside_code)

#把字符串全角转半角
def stringQ2B(ustring):
    """把字符串全角转半角"""
    return "".join([Q2B(uchar) for uchar in ustring])


#消息预处理
def msg_pre_process(msg):
    #大小写转化
    msg = str(msg).lower()
    
    #全角转半角
    msg = stringQ2B(msg)
    
    #去除空格 \xa0
    msg = msg.replace(' ','')
    msg =  "".join(msg.split())
    
    #同义词替换

    #词库新增
    
    #过滤停用词
    
    #分词
    segs = jieba.lcut(msg) 
    sentences = " ".join(segs)
    
    return sentences

text = pd.read_csv("./before_sell.csv", encoding='utf-8')
del text['所属公司']
del text['消息来源']
del text['目标类型']
del text['消息目标']
del text['新老访客']
del text['消息时间']
# 删除所有来源为客服的
result_index = []
for id, row in text.iterrows():
    if row['来源类型'] == '客服'or row['来源类型'] == '机器人':
        result_index.append(id)
text = text.drop(result_index, axis=0)
text = text.reset_index(drop=True)
text.head(10)

,会话ID,来源类型,消息内容
0,9884be2068f04d44aa8ce148b56374b0,访客,nbr1700路由器是否支持ipv6
1,9884be2068f04d44aa8ce148b56374b0,访客,eg2000是否支持ipv6
2,9884be2068f04d44aa8ce148b56374b0,访客,nbr1700路由器是否支持ipv6
3,NaN,NaN,NaN
4,a79dd015d89243b39fec087e4985fc4a,访客,人工
5,a79dd015d89243b39fec087e4985fc4a,访客,est310可以带几个400万摄像头
6,a79dd015d89243b39fec087e4985fc4a,访客,好没有
7,a79dd015d89243b39fec087e4985fc4a,访客,还没有
8,a79dd015d89243b39fec087e4985fc4a,访客,est310可以带几个400万摄像头
9,a79dd015d89243b39fec087e4985fc4a,访客,好的


In [17]:
final_need = []

for id, row in text.iterrows():
    dict = {}
#     mes_list = []
    msg = str(row['消息内容'])
    if len(msg) <= 5:
        continue
    if '该内容暂不支持显示'in msg:
        msg = msg.replace('该内容暂不支持显示','')
    
    dict["ID"] = row["会话ID"]
    dict["messege"] = msg
    
    
    final_need.append(dict)
print(len(final_need))
# for i in range(0,100):
#     print(final_need[i])

5362


In [19]:
# 没有分词的text
text = []
for id, sent in enumerate(final_need):
    text.append(sent['messege'])
print(text[:5])

['nbr1700路由器是否支持ipv6', 'eg2000是否支持ipv6', 'nbr1700路由器是否支持ipv6', 'est310可以带几个400万摄像头', 'est310可以带几个400万摄像头']


In [3]:
# 售前 所有问题的数据集
with open('question_v.0.1.txt','w',encoding='utf-8') as f:
    for id, sent in enumerate(final_need):
        msg = msg_pre_process(sent['messege'])
        f.write(msg+'\n')
os.path.getsize('./question_v.0.1.txt')


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\LLM\AppData\Local\Temp\jieba.cache
Loading model cost 0.472 seconds.
Prefix dict has been built successfully.


264617

In [38]:
# 售前 所有会话的数据集
import numpy
import pandas as pd 
import jieba
import jieba.analyse
import os

#单个字符 全角转半角
def Q2B(uchar):
    """单个字符 全角转半角"""
    inside_code = ord(uchar)
    if inside_code == 0x3000:
        inside_code = 0x0020
    else:
        inside_code -= 0xfee0
    if inside_code < 0x0020 or inside_code > 0x7e: #转完之后不是半角字符返回原来的字符
        return uchar
    return chr(inside_code)

#把字符串全角转半角
def stringQ2B(ustring):
    """把字符串全角转半角"""
    return "".join([Q2B(uchar) for uchar in ustring])


#消息预处理
def msg_pre_process(msg):
    #大小写转化
    msg = str(msg).lower()
    
    #全角转半角
    msg = stringQ2B(msg)
    
    #去除空格 \xa0
    msg = msg.replace(' ','')
    msg =  "".join(msg.split())
    
    #同义词替换

    #词库新增
    
    #过滤停用词
    
    #分词
    segs = jieba.lcut(msg) 
    sentences = " ".join(segs)
    
    return sentences

text = pd.read_csv("./before_sell.csv", encoding='utf-8')
del text['所属公司']
del text['消息来源']
del text['目标类型']
del text['消息目标']
del text['新老访客']
del text['消息时间']
text.head(5)
final_need = []

for id, row in text.iterrows():
    dict = {}
#     mes_list = []
    msg = str(row['消息内容'])
    if len(msg) <= 5:
        continue
    if '该内容暂不支持显示'in msg:
        msg = msg.replace('该内容暂不支持显示','')
    if msg == "":
        msg = "NULL"
    
    dict["ID"] = row["会话ID"]
    dict["messege"] = msg
    
    
    final_need.append(dict)

with open('question_v.1.1.txt','w',encoding='utf-8') as f:
    for id, sent in enumerate(final_need):
        if sent["messege"] == final_need[id-1]["messege"]:
            continue
        else:
            msg = msg_pre_process(sent['messege'])
            f.write(msg+'\n')
os.path.getsize('./question_v.1.1.txt')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\LLM\AppData\Local\Temp\jieba.cache
Loading model cost 0.704 seconds.
Prefix dict has been built successfully.


783445

In [4]:
# 想使用word2vec进行文本转向量
from gensim.models import word2vec
#sentences=[["a","b"],["b","c"] ... ]
sentences = word2vec.Text8Corpus("question_v.0.1.txt")  #text8为语料库文件名
model = word2vec.Word2Vec(sentences,sg=1,size=100,window=5,min_count=2,negative=3,sample=0.001,hs=1,workers=4)
model.save("w2v")
#对.sava保存的模型的加载：


In [5]:
model = word2vec.Word2Vec.load("w2v")
# 计算一个词的最近似的词：
# 只能计算词，不能计算句子
X = model.most_similar("区别",topn=10)	#计算与该 词最近似的词，topn指定排名前n的词
print(X)
# 计算两个词的相似度：
# Y = model.similarity("故障","问题")  
# print(Y)
# 获取词向量（有了这个不就意味着可以进行相关词语的加减等运算，虽然我不是太懂）：
# Z = model ['设备']
# print(Z)

[('x32ax3200', 0.9109019041061401), ('两种', 0.888174831867218), ('和', 0.8812873959541321), ('ax3200', 0.8739747405052185), ('eap', 0.8649444580078125), ('6210v2', 0.856655478477478), ('问下易', 0.8562938570976257), ('芯片', 0.852783739566803), ('更好', 0.8457072377204895), ('什么', 0.8409267663955688)]


<ipython-input-5-b85d2ca37a56>:4: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  X = model.most_similar("区别",topn=10)	#计算与该 词最近似的词，topn指定排名前n的词


In [6]:
# 加入售后数据看精度有无提升
text1 = pd.read_csv("./after_sell.csv", encoding='utf-8')
del text1['所属公司']
del text1['消息来源']
del text1['目标类型']
del text1['消息目标']
del text1['新老访客']
del text1['消息时间']
# 删除所有来源为客服的
result_index = []
for id, row in text1.iterrows():
    if row['来源类型'] == '客服'or row['来源类型'] == '机器人':
        result_index.append(id)
text1 = text1.drop(result_index, axis=0)
text1 = text1.reset_index(drop=True)

final_need1 = []

for id, row in text1.iterrows():
    dict = {}
#     mes_list = []
    msg = str(row['消息内容'])
    if len(msg) <= 5:
        continue
    if '该内容暂不支持显示'in msg:
        msg = msg.replace('该内容暂不支持显示','')
    
    dict["ID"] = row["会话ID"]
    dict["messege"] = msg
    
    
    final_need1.append(dict)
print(len(final_need1))

34968


In [7]:
with open('question_v.0.2.txt','w',encoding='utf-8') as f:
    for id, sent in enumerate(final_need1):
        msg = msg_pre_process(sent['messege'])
        f.write(msg+'\n')
os.path.getsize('./question_v.0.2.txt')

1429159

In [8]:
more_sentences = word2vec.Text8Corpus("question_v.0.2.txt")

#对.sava保存的模型的加载：
model = word2vec.Word2Vec.load("w2v")
model.train(more_sentences,epochs=3,total_examples=model.corpus_count)

(504021, 765117)

In [9]:
X = model.most_similar("区别",topn=10)	#计算与该 词最近似的词，topn指定排名前n的词
print(X)

[('6210v2', 0.8309206962585449), ('外形', 0.8017970323562622), ('啥', 0.7976089715957642), ('x32ax3200', 0.7946693301200867), ('小企业', 0.7911486029624939), ('好处', 0.7900347113609314), ('x32pro', 0.7849990129470825), ('两种', 0.7803089022636414), ('102', 0.7780659794807434), ('那锐捷', 0.7761869430541992)]


<ipython-input-9-4c339dfdca26>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  X = model.most_similar("区别",topn=10)	#计算与该 词最近似的词，topn指定排名前n的词


In [7]:
# 尝试用doc2vec提取高频问题
from gensim.models.doc2vec import Doc2Vec, LabeledSentence,TaggedDocument 

tagD = TaggedDocument
def dataTrain(sentence):
    trainset = []
    for i, text in enumerate(sentence):
        word_list = text.split(' ')
        l = len(word_list)
        word_list[l-1] = word_list[l-1].strip()
        document = tagD(word_list, tags=[i])
        trainset.append(document)
    return trainset
# 把整篇文档变为list
sentence_list = []
with open("./question_v.0.1.txt", 'r', encoding='utf-8') as f:
    for line in f:
        line = line.strip('\n')
        sentence_list.append(line)
# print(sentence_list[:10])
trainset = dataTrain(sentence_list)
print(trainset[:10])

[TaggedDocument(words=['nbr1700', '路由器', '是否', '支持', 'ipv6'], tags=[0]), TaggedDocument(words=['eg2000', '是否', '支持', 'ipv6'], tags=[1]), TaggedDocument(words=['nbr1700', '路由器', '是否', '支持', 'ipv6'], tags=[2]), TaggedDocument(words=['est310', '可以', '带', '几个', '400', '万', '摄像头'], tags=[3]), TaggedDocument(words=['est310', '可以', '带', '几个', '400', '万', '摄像头'], tags=[4]), TaggedDocument(words=['客户', '有', '三个', 'iptv', ',', '有', '面板', 'apeap102', ',', '网关', 'eg210g', '-', 'e'], tags=[5]), TaggedDocument(words=['这', '三个', '网关', '都', '可以', '用', '吗'], tags=[6]), TaggedDocument(words=['这', '三个', 'iptv', '都', '可以', '用', '吗'], tags=[7]), TaggedDocument(words=['eap102', ',', 'eg210g', '-', 'e'], tags=[8]), TaggedDocument(words=['睿易', 'app', '的', '中级', '工程师'], tags=[9])]


In [24]:
def train(trainset, size=500): # size 是特征向量的维度
    model = Doc2Vec(trainset, min_count=1,window=5,size=size,sample=1e-3,negative=5,workers=4)
    model.train(trainset,total_examples=model.corpus_count,epochs=10)
    return model

model_dm = train(trainset)

C:\Users\LLM\Anaconda3\lib\site-packages\gensim\models\doc2vec.py:319: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [25]:
"""模型训练完毕以后，可以预测新的句子的向量
这里用gensim里Doc2Vec.infer_vector()预测新的句子，根据网上经验，alpha（学习步长）设置小一些，迭代次数设置大一些。
找到训练样本中与这个句子最相近的10个句子。可以看到训练出来的结果与测试的新句子是有关联的。"""

str1 = 'nbr1700 路由器 是否 支持 ipv6'
test_text = str1.split(' ')
inferred_vector = model_dm.infer_vector(doc_words=test_text,alpha=0.025,steps=300)

sims = model_dm.docvecs.most_similar([inferred_vector],topn=10)
# print(sims)
for count,sim in sims:
    sentence = text[count]
    words = ''
    for word in sentence:
        words = words + word + ' '
    print(words, sim, len(sentence))


请 问 这 款 交 换 机 是 否 支 持 端 口 镜 像 功 能 ？ 如 何 设 置 ？  0.9854362607002258 23
a c ： r g - e g 2 1 0 g - p  0.9475691914558411 14
 0.9321928024291992 0
最 大 支 持 多 少 并 发 用 户 ？  0.9306647777557373 11
路 由 ： r g - e g 2 1 0 g - e 交 换 机 ： r g - e s 2 0 9 g c - p a p ： r g - r a p 2 2 0 0 （ e ）  0.93006432056427 46
兼 容 设 备 型 号 ： n b r 6 1 3 5 - e  0.9294251203536987 16
n b r 7 0 0 系 列 / 8 0 0 g w _ 无 线 功 能 配 置 （ 含 w i f i 、 a p 管 理 、 网 优 等 ）  0.9235466718673706 37
请 问 n b r 6 2 0 5 e 支 持 i p v 6 吗 ？  0.9192467331886292 18
您 好 我 有 俩 个 方 案 您 是 专 家 帮 我 看 看 哪 个 更 适 合 家 庭 1 、 r g - e a p 2 6 2 ( g ) ： 1 台 、 r g - e g 1 0 5 g - p - e ： 1 台 、 r g - e a p 1 0 2 ： 2 台 。 第 二 套 ： r g - e a p 2 6 2 ( g ) ： 1 台 、 r g - e g 1 0 5 g - p - e ： 1 台 、 r g - e a p 1 6 2  0.9155741930007935 117
e a p 1 6 2 （ g ） 支 持 i p v 6 吗  0.9087147116661072 16


In [4]:
#keybert 提取关键词
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2") # 换预训练模型
kw_model = KeyBERT(model=sentence_model)
print(kw_model)

In [7]:
# 使用所有问题的数据集
doc = ""
with open("./question_v.0.1.txt","r",encoding="utf-8") as f:
    for line in f:
        line = line.strip('\n')
        doc += " " + line
keywords = kw_model.extract_keywords(doc,keyphrase_ngram_range=(1,2),stop_words=None,top_n=50)
#keyphrase_ngram_range参数：
# (1, 1) 只单个词，(2, 2) 考虑词组， 如出现有意义的词组 "抗美援朝 纪念日" (1, 2) 同时考虑以上两者情况
print(keywords)
print("\n")
# 使用最大相关度(maximum Margin Relevance, MMR)来创建同样基于余弦相似度的关键字/关键短语
#
keywords = kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 2), stop_words=None, 
                              use_mmr=True, diversity=0.7,top_n=50)
print("使用最大相关度（高多样性）：",keywords)
print("\n")
keywords = kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 2), stop_words=None, 
                              use_mmr=True, diversity=0.2,top_n=50)
print("使用最大相关度（低多样性）：",keywords)
print("\n")

# 使用最大和相似
keywords = kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 2), stop_words=None, 
                              use_maxsum=True, nr_candidates=20, top_n=10)
print("使用最大和相似：",keywords)

[('ipv6 网络', 0.734), ('ipv6 est310', 0.7207), ('ipv6 网关', 0.7131), ('ipv6 eg2000', 0.7105), ('ip1000mb 带宽', 0.7089), ('ipv6 nbr1700', 0.7016), ('ipv6 双栈', 0.6984), ('配置 ipv6', 0.6904), ('ipv6 协议', 0.6901), ('支持 ipv4', 0.6893), ('ipv6 eap162', 0.6892), ('支持 ipv6', 0.6882), ('ipv6 支持', 0.6843), ('透传 ipv6', 0.6788), ('ipv4 ipv6', 0.6756), ('eap212 宽带', 0.6702), ('ipv6 可以', 0.6657), ('ipv6telnet 15', 0.6657), ('iptv102', 0.6649), ('ipv4', 0.6618), ('系列 ipv6', 0.6597), ('ipv6telnet', 0.6582), ('ipv4 ipv6acl', 0.6568), ('iptv 宽带', 0.6545), ('14 ipv6', 0.6537), ('ipsecvpn 带宽', 0.6526), ('iptv102 支持', 0.6515), ('aclv6 ipv6telnet', 0.6464), ('ip1000mb', 0.6435), ('ipv6 请教', 0.643), ('ipv6 nbc', 0.6422), ('ipv6 特性', 0.6413), ('ipv6 nbr', 0.6354), ('ipv6acl 支持', 0.633), ('网络线 iptv', 0.6283), ('开启 ipv6', 0.6277), ('ipv6', 0.6273), ('ipv6 rg', 0.6268), ('固定 ip1000mb', 0.6267), ('ipv6 那么', 0.6256), ('ipv6 对于', 0.6248), ('iptv 传输', 0.6185), ('ipv6 nbr6', 0.6175), ('上网 nbr800g', 0.6175), ('三个 iptv', 0

In [6]:

# 使用对话文本作为数据集
doc = ""
with open("./question_v.1.1.txt","r",encoding="utf-8") as f:
    for line in f:
#         print(1)
        line = line.strip('\n')
#         print(1)
        doc += " " + line
#         print(1)
        if line == 'null': 
            keywords = kw_model.extract_keywords(doc,keyphrase_ngram_range=(1,1),stop_words=None,top_n=1)
            with open("./question_v.1.2.2.txt", "a", encoding="utf-8") as f1:
                for text in keywords:
#                     print(text[1])
                    if text[0] == 'null':
                        continue
                    if(text[1] > 0.6):
                        f1.write(text[0])
                        f1.write("\n")
                
            line.strip('\n')
            doc = ""

              
# keywords = kw_model.extract_keywords(doc,keyphrase_ngram_range=(1,2),stop_words=None,top_n=50)
# print(keywords)
# print("\n")
# keywords = kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 2), stop_words=None, 
#                               use_mmr=True, diversity=0.7,top_n=50)
# print("使用最大相关度（高多样性）：",keywords)
# print("\n")
# keywords = kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 2), stop_words=None, 
#                               use_mmr=True, diversity=0.2,top_n=50)
# print("使用最大相关度（低多样性）：",keywords)
# print("\n")

# 使用最大和相似
# keywords = kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 2), stop_words=None, 
#                               use_maxsum=True, nr_candidates=20, top_n=10)
# print("使用最大和相似：",keywords)

In [10]:
# 文本聚类

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn import metrics
from time import time
from sklearn.decomposition import PCA
from collections import Counter
import numpy as np

import re
import jieba
import jieba.analyse
from sklearn.cluster import DBSCAN

stop_words = ['“', '右边', 'nbr6205e', '1800gxpro', 'eap162', '还有', '必须', '可能', 'qd', 'sd', 'hsip', 'gov',
               '然后', '要求', '一为', '我想', '新', '有些', '其中', '两个', '；', '≥',
               '，', 'wall1600', 'eap162g', '包', '提供', '销售', '找', '客户', '只能', 'ｉ', '办法', '调整', 'nbs1850gc', 's1826g',
               'es226gc', '买', '联系', '打', '1000m', '这些', 'eg105g', 'nbs5200', '24sfp', '8gt4xs', '说', 'rap630',
               '\u3000', '公司', '时候', '+', 'rap220v2nbr6135', '指', '一台', '处理', '300', '总', 'rg', 'nbr6135', '24gt4sfp',
               'p', 'nbs5710', '404', '麻烦', '好像', '*', '1.1', '1t', '24', 'qq', '%', '4', '_', '看看', '获取', '正常', '！',
               '远程', '2', '如何', '开头', '产品型号', '标贴', '、',
               '系列', '工程师', '~', '吧', '当前', '哈', '售后', '一般',
               '我','EW1800', 'EW1800G','pro','NBR','想要','5628xg','地','早就','解决','EG105G','NBS2026G','P','E','EAP','218GC','啥事','看到',
               '二维码','先加','起来','\xa0','rap220','被','去','ES126GC','LP','210','不停','nbs2028G','RG','AC128','EAP602',' '
               '的',
               '吗',
               '想',
               '了',
               '是',
               '？',
               '怎么',
               '这个',
               '有',
               #  '设置',
               '可以',
               '在',
               # '是不是',
               '问题',
               '但是',
               '.',
               #  '设备',
               '一个',
               #  '访问',
               '不',
               '一下',
               '。',
               '做',
               '-',
               '你好',
               '咨询',
               '呢',
               '能',
               '就是',
               '到',
               '问',
               '什么',
               '现在',
               '上',
               '都',
               '和',
               '就',
               '要',
               '需要',
               '下',
               '把',
               '这边',
               '知道',
               '你',
               '用',
               '这',
               '没',
               '走',
               '没有',
               '会', '*', '%', '4', '_', '看看', '获取', '正常', '！', '远程', '2', '如何', '开头',  '标贴', '、',
               '系列', '工程师', '~', '吧', '当前',  '哈', '售后', '一般',
               '设备', '稍等', '（', '请', '转接', '：', ':', '查看', '正在', '帮', '锐捷',
               '您', '显示', '该', '内容', '暂', '=', '您好', '嗯',  '/', '）', '为',
              '，', '的', '我', '吗', '是', '？', '了', '想', '可以', '这个', '怎么', '有', '访问', '问题'
    , '在', '但是', '-', '一下', '不', '.', '能', '请问',  '上', '一个', '。'
    , '现在', '问', '呢', '做', '和', '就', '这边', '需要', '就是', '下', '也', '会',
               '没有', '要', '到', '都', 'xx', '你好', '那', '我们', '这样', '咨询', '什么', '如果', '192.168', '把', '你'
    , '里面', '通过', '这', '用', '看', '好', '没', '？', '段', '给', '个', '走'
    , '?', '后', '还是', '那个', '的话', '还', '实现', '1', 'e', '其他', '让', '你们', '对', '用户', '问下', '谢谢', '啊', '(', ')', '）' ]


f = open(r'./question_v.1.1.txt','r', encoding="utf-8")
topic = list(f)
f.close()

vectorizer = TfidfVectorizer(sublinear_tf=True, stop_words=stop_words)
#统计每个词语的tf-idf权值
transformer = TfidfTransformer()
# 第一个fit_transform是计算tf-idf 第二个fit_transform是将文本转为词频矩阵
tfidf = transformer.fit_transform(vectorizer.fit_transform(topic))
# 获取词袋模型中的所有词语
word = vectorizer.get_feature_names() # list
word_arr = np.array(word)
# print(word_arr)
# 将tf-idf矩阵抽取出来，元素w[i][j]表示j词在i类文本中的tf-idf权重
weight = tfidf.toarray() # tuple

pca = PCA(n_components=15) 
weight = pca.fit_transform(weight)  # 载入N维, tuple
#查看特征大小
# print ('Features length: ' + str(len(word)))

# 将list转为dataframe
import pandas
from pandas.core.frame import DataFrame

temp = {
    "ID":0,
    "keyword": topic
}
data = DataFrame(temp)
data.head(10)

C:\Users\LLM\Anaconda3\envs\ruijie\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['168', '192', '218gc', 'ac128', 'eap', 'eap602', 'es126gc', 'ew1800', 'ew1800g', 'lp', 'nbr', 'nbs2026g', 'nbs2028g'] not in stop_words.
  warnings.warn(


,ID,keyword
0,0,nbr1700 路由器 是否 支持 ipv6\n
1,0,"抱歉 哈 , 当前 是 行业 售前 , 您 咨询 的 问题 属于 锐捷 睿 易 售前 问题 ..."
2,0,"您好 , 锐捷 睿 易\n"
3,0,eg2000 是否 支持 ipv6\n
4,0,nbr1700 路由器 是否 支持 ipv6\n
5,0,nbr 全系列 是否 支持 ipv6 ? 答 : nbr 全系列 产品 不 支持 ipv6 ...
6,0,"不好意思 , nbr 全系列 不 支持 的\n"
7,0,"您好 , 当前 是 锐捷 售后 , 请问 您 咨询 的 产品型号 是 多少 呢 ? ( 可以..."
8,0,est310 可以 带 几个 400 万 摄像头\n
9,0,"您好 , 请问 您 是否 购买 设备 了 呢 ?\n"


In [11]:
import hdbscan


clusterer = hdbscan.HDBSCAN(min_cluster_size=3)
cluster_labels = clusterer.fit_predict(weight)
data["ID"] = cluster_labels
label_dict = {}
for i in list(set(list(data["ID"]))):
    tmp_df = data[data.ID == i]['keyword']
    len_tmp = len(tmp_df)
    tmp_words =[]
    for j in tmp_df:
        tmp_words.extend(j.split(' '))
    key_word = kw_model.extract_keywords(tmp_words,keyphrase_ngram_range=(1,1),stop_words=None,top_n=1) # 提取关键词
    key_word = ' '.join(k[0][0] for k in key_word)
    label_dict[i] = key_word
#     print('类目序号 :{label},数据共计{len_tmp}，关键字:{key_word}'.format(label = i,len_tmp= len_tmp,key_word=key_word))

C:\Users\LLM\Anaconda3\envs\ruijie\lib\site-packages\keybert\_model.py:130: UserWarning: Although extracting keywords for multiple documents is faster than iterating over single documents, it requires significantly more memory to hold all word embeddings. Use this at your own discretion!
  warnings.warn("Although extracting keywords for multiple documents is faster "
532it [00:00, 1515.33it/s]
C:\Users\LLM\Anaconda3\envs\ruijie\lib\site-packages\keybert\_model.py:130: UserWarning: Although extracting keywords for multiple documents is faster than iterating over single documents, it requires significantly more memory to hold all word embeddings. Use this at your own discretion!
  warnings.warn("Although extracting keywords for multiple documents is faster "
1924it [00:00, 2952.68it/s]
C:\Users\LLM\Anaconda3\envs\ruijie\lib\site-packages\keybert\_model.py:130: UserWarning: Although extracting keywords for multiple documents is faster than iterating over single documents, it requires sign

56it [00:00, 1713.07it/s]
C:\Users\LLM\Anaconda3\envs\ruijie\lib\site-packages\keybert\_model.py:130: UserWarning: Although extracting keywords for multiple documents is faster than iterating over single documents, it requires significantly more memory to hold all word embeddings. Use this at your own discretion!
  warnings.warn("Although extracting keywords for multiple documents is faster "
391it [00:00, 2424.38it/s]
C:\Users\LLM\Anaconda3\envs\ruijie\lib\site-packages\keybert\_model.py:130: UserWarning: Although extracting keywords for multiple documents is faster than iterating over single documents, it requires significantly more memory to hold all word embeddings. Use this at your own discretion!
  warnings.warn("Although extracting keywords for multiple documents is faster "
24it [00:00, 15721.27it/s]
C:\Users\LLM\Anaconda3\envs\ruijie\lib\site-packages\keybert\_model.py:130: UserWarning: Although extracting keywords for multiple documents is faster than iterating over single do

47it [00:00, 2581.99it/s]
C:\Users\LLM\Anaconda3\envs\ruijie\lib\site-packages\keybert\_model.py:130: UserWarning: Although extracting keywords for multiple documents is faster than iterating over single documents, it requires significantly more memory to hold all word embeddings. Use this at your own discretion!
  warnings.warn("Although extracting keywords for multiple documents is faster "
210it [00:00, 4538.21it/s]
C:\Users\LLM\Anaconda3\envs\ruijie\lib\site-packages\keybert\_model.py:130: UserWarning: Although extracting keywords for multiple documents is faster than iterating over single documents, it requires significantly more memory to hold all word embeddings. Use this at your own discretion!
  warnings.warn("Although extracting keywords for multiple documents is faster "
46it [00:00, ?it/s]
C:\Users\LLM\Anaconda3\envs\ruijie\lib\site-packages\keybert\_model.py:130: UserWarning: Although extracting keywords for multiple documents is faster than iterating over single documents

119it [00:00, 1301.87it/s]
C:\Users\LLM\Anaconda3\envs\ruijie\lib\site-packages\keybert\_model.py:130: UserWarning: Although extracting keywords for multiple documents is faster than iterating over single documents, it requires significantly more memory to hold all word embeddings. Use this at your own discretion!
  warnings.warn("Although extracting keywords for multiple documents is faster "
50it [00:00, 982.03it/s]
C:\Users\LLM\Anaconda3\envs\ruijie\lib\site-packages\keybert\_model.py:130: UserWarning: Although extracting keywords for multiple documents is faster than iterating over single documents, it requires significantly more memory to hold all word embeddings. Use this at your own discretion!
  warnings.warn("Although extracting keywords for multiple documents is faster "
74it [00:00, 4423.30it/s]
C:\Users\LLM\Anaconda3\envs\ruijie\lib\site-packages\keybert\_model.py:130: UserWarning: Although extracting keywords for multiple documents is faster than iterating over single docu

12it [00:00, 1107.75it/s]
C:\Users\LLM\Anaconda3\envs\ruijie\lib\site-packages\keybert\_model.py:130: UserWarning: Although extracting keywords for multiple documents is faster than iterating over single documents, it requires significantly more memory to hold all word embeddings. Use this at your own discretion!
  warnings.warn("Although extracting keywords for multiple documents is faster "
33it [00:00, 1924.90it/s]
C:\Users\LLM\Anaconda3\envs\ruijie\lib\site-packages\keybert\_model.py:130: UserWarning: Although extracting keywords for multiple documents is faster than iterating over single documents, it requires significantly more memory to hold all word embeddings. Use this at your own discretion!
  warnings.warn("Although extracting keywords for multiple documents is faster "
32it [00:00, ?it/s]
C:\Users\LLM\Anaconda3\envs\ruijie\lib\site-packages\keybert\_model.py:130: UserWarning: Although extracting keywords for multiple documents is faster than iterating over single documents,

...

89it [00:00, 3067.83it/s]
C:\Users\LLM\Anaconda3\envs\ruijie\lib\site-packages\keybert\_model.py:130: UserWarning: Although extracting keywords for multiple documents is faster than iterating over single documents, it requires significantly more memory to hold all word embeddings. Use this at your own discretion!
  warnings.warn("Although extracting keywords for multiple documents is faster "
47it [00:00, 1174.19it/s]
C:\Users\LLM\Anaconda3\envs\ruijie\lib\site-packages\keybert\_model.py:130: UserWarning: Although extracting keywords for multiple documents is faster than iterating over single documents, it requires significantly more memory to hold all word embeddings. Use this at your own discretion!
  warnings.warn("Although extracting keywords for multiple documents is faster "
32it [00:00, 929.30it/s]
C:\Users\LLM\Anaconda3\envs\ruijie\lib\site-packages\keybert\_model.py:130: UserWarning: Although extracting keywords for multiple documents is faster than iterating over single docum

23it [00:00, 1370.08it/s]
C:\Users\LLM\Anaconda3\envs\ruijie\lib\site-packages\keybert\_model.py:130: UserWarning: Although extracting keywords for multiple documents is faster than iterating over single documents, it requires significantly more memory to hold all word embeddings. Use this at your own discretion!
  warnings.warn("Although extracting keywords for multiple documents is faster "
71it [00:00, 1985.98it/s]
C:\Users\LLM\Anaconda3\envs\ruijie\lib\site-packages\keybert\_model.py:130: UserWarning: Although extracting keywords for multiple documents is faster than iterating over single documents, it requires significantly more memory to hold all word embeddings. Use this at your own discretion!
  warnings.warn("Although extracting keywords for multiple documents is faster "
75it [00:00, 1849.11it/s]
C:\Users\LLM\Anaconda3\envs\ruijie\lib\site-packages\keybert\_model.py:130: UserWarning: Although extracting keywords for multiple documents is faster than iterating over single docu

17it [00:00, 1087.35it/s]
C:\Users\LLM\Anaconda3\envs\ruijie\lib\site-packages\keybert\_model.py:130: UserWarning: Although extracting keywords for multiple documents is faster than iterating over single documents, it requires significantly more memory to hold all word embeddings. Use this at your own discretion!
  warnings.warn("Although extracting keywords for multiple documents is faster "
30it [00:00, 1070.63it/s]
C:\Users\LLM\Anaconda3\envs\ruijie\lib\site-packages\keybert\_model.py:130: UserWarning: Although extracting keywords for multiple documents is faster than iterating over single documents, it requires significantly more memory to hold all word embeddings. Use this at your own discretion!
  warnings.warn("Although extracting keywords for multiple documents is faster "
13it [00:00, ?it/s]
C:\Users\LLM\Anaconda3\envs\ruijie\lib\site-packages\keybert\_model.py:130: UserWarning: Although extracting keywords for multiple documents is faster than iterating over single documents,

33it [00:00, 819.09it/s]
C:\Users\LLM\Anaconda3\envs\ruijie\lib\site-packages\keybert\_model.py:130: UserWarning: Although extracting keywords for multiple documents is faster than iterating over single documents, it requires significantly more memory to hold all word embeddings. Use this at your own discretion!
  warnings.warn("Although extracting keywords for multiple documents is faster "
75it [00:00, 1548.73it/s]
C:\Users\LLM\Anaconda3\envs\ruijie\lib\site-packages\keybert\_model.py:130: UserWarning: Although extracting keywords for multiple documents is faster than iterating over single documents, it requires significantly more memory to hold all word embeddings. Use this at your own discretion!
  warnings.warn("Although extracting keywords for multiple documents is faster "
18it [00:00, 654.63it/s]
C:\Users\LLM\Anaconda3\envs\ruijie\lib\site-packages\keybert\_model.py:130: UserWarning: Although extracting keywords for multiple documents is faster than iterating over single docume

55it [00:00, 3287.49it/s]
C:\Users\LLM\Anaconda3\envs\ruijie\lib\site-packages\keybert\_model.py:130: UserWarning: Although extracting keywords for multiple documents is faster than iterating over single documents, it requires significantly more memory to hold all word embeddings. Use this at your own discretion!
  warnings.warn("Although extracting keywords for multiple documents is faster "
43it [00:00, 2391.91it/s]
C:\Users\LLM\Anaconda3\envs\ruijie\lib\site-packages\keybert\_model.py:130: UserWarning: Although extracting keywords for multiple documents is faster than iterating over single documents, it requires significantly more memory to hold all word embeddings. Use this at your own discretion!
  warnings.warn("Although extracting keywords for multiple documents is faster "
98it [00:00, 3384.12it/s]
C:\Users\LLM\Anaconda3\envs\ruijie\lib\site-packages\keybert\_model.py:130: UserWarning: Although extracting keywords for multiple documents is faster than iterating over single docu

In [16]:
import pandas as pd

# output_path='D:/锐捷网络/datasets/final_0.csv' # 使用所有问题数据集产生的结果
output_path='D:/锐捷网络/datasets/final_1.csv' # 使用所有对话数据集产生的结果
df = pd.DataFrame(list(label_dict.items()))
df.to_csv(outputpath,sep=',',encoding="utf_8_sig",index=False,header=False) 
